In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import os
from PIL import Image
import random
import numpy as np
import cv2
import glob
import PIL.ImageOps
import shutil

In [ ]:
patH = input('작업폴더 경로를 입력해주세요 : ')     # /content/gdrive/MyDrive/Project/image/doro_contest
%cd {patH}

작업폴더 경로를 입력해주세요 : /content/gdrive/MyDrive/Project/image/doro_contest
/content/gdrive/MyDrive/Project/image/doro_contest


# train, test jpg 파일 나누기

In [ ]:
foldername = input('처리하실 foldername 을 입력해주세요 (한자리 숫자는 꼭 0을 같이 입력해주세요!)>> ')
file_path = patH + '/road/'+foldername+'/new/'

n_path = file_path + '1/'
y_path = file_path + '0/'

n_filenames = os.listdir(n_path)
y_filenames = os.listdir(y_path)

file_names = n_filenames + y_filenames
total_len = len(file_names)
test_len = int(total_len * 0.2)
test_filename = random.sample(file_names, test_len)
train_filename = file_names.copy()
for i in test_filename : 
  train_filename.remove(i)

처리하실 foldername 을 입력해주세요 (한자리 숫자는 꼭 0을 같이 입력해주세요!)>> 38


In [ ]:
os.mkdir(file_path+'test')
os.mkdir(file_path+'test'+'/0')
os.mkdir(file_path+'test'+'/1')
os.mkdir(file_path+'train')
os.mkdir(file_path+'train'+'/0')
os.mkdir(file_path+'train'+'/1')

for i in test_filename : 
  if i in n_filenames :
    src = n_path 
    dir = file_path + 'test/1/'
    shutil.move(src + i, dir + i)
  else :
    src = y_path 
    dir = file_path + 'test/0/'
    shutil.move(src + i, dir + i)

for i in train_filename : 
  if i in n_filenames :
    src = n_path 
    dir = file_path + 'train/1/'
    shutil.move(src + i, dir + i)
  else :
    src = y_path 
    dir = file_path + 'train/0/'
    shutil.move(src + i, dir + i)

In [ ]:
train_filepath = []
test_filepath = []

train_path = file_path + 'train/'
test_path = file_path + 'test/'

for i in train_filename : 
  if i in n_filenames : 
    train_filepath.append(train_path+'1/'+i)
  else : 
    train_filepath.append(train_path+'0/' + i)

for i in test_filename : 
  if i in n_filenames : 
    test_filepath.append(test_path+'1/'+i)
  else : 
    test_filepath.append(test_path+'0/' + i)

# augmentation

In [ ]:
num_augmented_images = 300   # 새로 만들 이미지의 개수
augment_cnt =1

for i in range(1, num_augmented_images) : 
  change_picture_index = random.randrange(1, len(train_filename) - 1)
  print(change_picture_index)
  print(train_filename[change_picture_index])
  file_name = train_filename[change_picture_index]
  origin_image_path = train_filepath[change_picture_index]
  image = Image.open(origin_image_path)
  random_augment = random.randrange(1, 4)   #1~3의 무작위 수 결정 ( 1: 좌우반전, 2 : 랜덤하게 -20 ~ 20 회전, 3 : 가우스 노이즈 추가)

  if(random_augment == 1):
      #이미지 좌우 반전
      print("invert")
      inverted_image = image.transpose(Image.FLIP_LEFT_RIGHT)
      inverted_image.save(origin_image_path[:-4] + '_inverted_' + str(augment_cnt) + '.jpg')
      
  elif(random_augment == 2):
      #이미지 기울이기
      print("rotate")
      rotated_image = image.rotate(random.randrange(-20, 20))
      rotated_image.save(origin_image_path[:-4] + '_rotated_' + str(augment_cnt) + '.jpg')
      
  elif(random_augment == 3):
      #노이즈 추가하기
      img = cv2.imread(origin_image_path)
      print("noise")
      row,col,ch= img.shape
      mean = 0
      var = 0.1
      sigma = var**0.5
      gauss = np.random.normal(mean,sigma,(row,col,ch))
      gauss = gauss.reshape(row,col,ch)
      noisy_array = img + gauss
      noisy_image = Image.fromarray(np.uint8(noisy_array)).convert('RGB')
      noisy_image.save(origin_image_path[:-4] + '_noiseAdded_' + str(augment_cnt) + '.jpg')
      
  augment_cnt += 1

152
3_38_2_1_05_2_20211110_000096333_192.jpg
invert
124
3_38_2_1_05_1_20211108_000088218_157.jpg
invert
256
3_38_1_1_05_1_20211110_000096439_161.jpg
rotate
306
3_38_1_1_01_1_20211111_000093560_26.jpg
rotate
120
3_38_2_1_05_1_20211105_000089478_155.jpg
rotate
35
3_38_2_1_01_2_20211113_000092211_47.jpg
noise
112
3_38_2_1_05_1_20211105_000086845_148.jpg
rotate
235
3_38_1_1_05_1_20211105_000086859_126.jpg
noise
8
3_38_2_1_01_1_20211105_000092591_18.jpg
invert
121
3_38_2_1_05_1_20211105_000089472_154.jpg
noise
181
3_38_1_1_04_1_20211030_000078479_64.jpg
invert
61
3_38_2_1_03_3_20211106_000084951_79.jpg
rotate
83
3_38_2_1_04_2_20211103_000081615_109.jpg
rotate
136
3_38_2_1_05_2_20211102_000088260_170.jpg
rotate
18
3_38_2_1_01_1_20211113_000092943_28.jpg
rotate
292
3_38_1_1_01_1_20211102_000080424_7.jpg
invert
118
3_38_2_1_05_1_20211105_000089451_151.jpg
noise
223
3_38_1_1_05_1_20211105_000085865_117.jpg
invert
92
3_38_2_1_04_2_20211109_000089909_119.jpg
noise
143
3_38_2_1_05_2_20211104_00008

# 데이터 정제

In [ ]:
foldername = input('처리하실 foldername 을 입력해주세요 (한자리 숫자는 꼭 0을 같이 입력해주세요!)>> ')

transform = transforms.Compose([transforms.Resize((240,240)),   # b1 -> 240 (width)   # 사용할 모델이 따라 다르게 적용
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
train_dataset = datasets.ImageFolder(f'{patH}/road/'+foldername+'/new/train', transform = transform)
test_dataset = datasets.ImageFolder(f'{patH}/road/'+foldername+'/new/test', transform = transform)

# batch_size // 하이퍼파라미터 --> 
batch_size = int(input('batch_size를 입력해주세요 >>'))

처리하실 foldername 을 입력해주세요 (한자리 숫자는 꼭 0을 같이 입력해주세요!)>> 16
batch_size를 입력해주세요 >>32


In [ ]:
train_len = len(train_dataset)
test_len = len(test_dataset)

print('학습 데이터셋 크기:', len(train_dataset))
print('테스트 데이터셋 크기:', len(test_dataset))

class_names = train_dataset.classes
print('클래스:', class_names)

학습 데이터셋 크기: 615
테스트 데이터셋 크기: 79
클래스: ['0', '1']


In [ ]:
train_set = DataLoader(dataset = train_dataset, shuffle = True, batch_size = batch_size)
test_set = DataLoader(dataset = test_dataset, shuffle = True, batch_size = batch_size)

# EfficientNet

In [ ]:
import torch
import torch.nn as nn
from math import ceil

base_model = [
  # expand_ratio, channels, repeats, stride, kernel_size
  [1, 16, 1, 1, 3],
  [6, 24, 2, 2, 3],
  [6, 40, 2, 2, 5],
  [6, 80, 3, 2, 3],
  [6, 112, 3, 1, 5],
  [6, 192, 4, 2, 5],
  [6, 320, 1, 1, 3]
]

phi_values = {
    # tuple of : (phi_value, resolution, drop_rate)
    "b0" : (0, 224, 0.2),  #alpha, beta, gamma, depth = alpha ** phi
    "b1" : (0.5, 240, 0.2),
    "b2" : (1, 260, 0.3),
    "b3" : (2, 300, 0.3),
    "b4" : (3, 380, 0.4),
    "b5" : (4, 456, 0.4),
    "b6" : (5, 528, 0.5),
    "b7" : (6, 600, 0.5),
}

class CNNBlock(nn.Module) : 
  def __init__(self, in_channels, out_channels, kernel_size, stride, padding, groups = 1) : 
    super(CNNBlock, self).__init__()
    self.cnn = nn.Conv2d(
        in_channels, 
        out_channels,
        kernel_size,
        stride,
        padding,
        groups = groups,
        bias = False,
    )
    self.bn = nn.BatchNorm2d(out_channels)
    self.silu = nn.SiLU()  # SILU <-> Swish
  def forward(self, x) : 
    return self.silu(self.bn(self.cnn(x)))

class SqueezeExcitation(nn.Module) : 
  def __init__(self, in_channels, reduced_dim) : 
    super(SqueezeExcitation, self).__init__()
    self.se = nn.Sequential(
        nn.AdaptiveAvgPool2d(1),  # C x H x @ -> C x 1 x 1
        nn.Conv2d(in_channels, reduced_dim, 1),
        nn.SiLU(),
        nn.Conv2d(reduced_dim, in_channels, 1),
        nn.Sigmoid()
    )
  def forward(self, x) : 
    return x * self.se(x)

class InvertedResidualBlock(nn.Module) : 
  def __init__(
      self, 
      in_channels, 
      out_channels, 
      kernel_size, 
      stride, 
      padding, 
      expand_ratio, 
      reduction = 4,   # squeeze excitation
      survival_prob = 0.8,  # for stochastic depth
      ) : 
      super(InvertedResidualBlock, self).__init__()
      self.survival_prob = 0.8
      self.use_residual = in_channels == out_channels and stride == 1
      hidden_dim = in_channels * expand_ratio
      self.expand = in_channels != hidden_dim
      reduced_dim = int(in_channels / reduction)

      if self.expand : 
        self.expand_conv = CNNBlock(
            in_channels, hidden_dim, kernel_size = 3, stride = 1, padding = 1,
        )
      
      self.conv = nn.Sequential(
          CNNBlock(
              hidden_dim, hidden_dim, kernel_size, stride, padding, groups = hidden_dim,
          ),
          SqueezeExcitation(hidden_dim, reduced_dim),
          nn.Conv2d(hidden_dim, out_channels, 1, bias = False),
          nn.BatchNorm2d(out_channels),

      )
    
  def stochastic_depth(self, x) : 
    if not self.training : 
      return x
    
    binary_tensor = torch.rand(x.shape[0], 1, 1, 1, device = x.device) < self.survival_prob
    return torch.div(x, self.survival_prob) * binary_tensor

  def forward(self, inputs) : 
    x = self.expand_conv(inputs) if self.expand else inputs
    if self.use_residual : 
      return self.stochastic_depth(self.conv(x)) + inputs
    else : 
      return self.conv(x)


class EfficientNet(nn.Module) : 
  def __init__(self, version, num_classes) : 
    super(EfficientNet, self).__init__()
    width_factor, depth_factor, dropout_rate = self.calculate_factors(version)
    last_channels = ceil(1280 * width_factor)
    self.pool = nn.AdaptiveAvgPool2d(1)
    self.features = self.create_features(width_factor, depth_factor, last_channels)
    self.classifier = nn.Sequential(
        nn.Dropout(dropout_rate),
        nn.Linear(last_channels, num_classes),

    )

  def calculate_factors(self, version, alpha = 1.2, beta = 1.1) : 
    phi, res, drop_rate = phi_values[version]
    depth_factor = alpha ** phi
    width_factor = beta ** phi
    return width_factor, depth_factor, drop_rate

  def create_features(self, width_factor, depth_factor, last_channels) : 
    channels = int(32 * width_factor)
    features = [CNNBlock(3, channels, 3, stride = 2, padding = 1)]
    in_channels = channels

    for expand_ratio, channels, repeats, stride, kernel_size in base_model :     
      out_channels = 4 * ceil(int(channels * width_factor) / 4)
      layers_repeats = ceil(repeats * depth_factor)

      for layer in range(layers_repeats) : 
        features.append(
            InvertedResidualBlock(
                in_channels,
                out_channels,
                expand_ratio = expand_ratio,
                stride = stride if layer == 0 else 1,
                kernel_size = kernel_size,
                padding = kernel_size//2,  # if k = 1 : pad = 0, k = 3 : pad = 1, k = 5 : pad = 2
            )
        )
        in_channels = out_channels

    features.append(CNNBlock(in_channels, last_channels, kernel_size = 1, stride = 1, padding = 0))

    return nn.Sequential(*features)
  
  def forward(self, x) : 
    x = self.pool(self.features(x))
    return self.classifier(x.view(x.shape[0], -1))



# device = "cuda" if torch.cuda.is_available() else "cpu"
version = input("Efficient Net 모델을 입력해주세요(b0 ~ b7) >> ")
phi, res, drop_rate = phi_values[version]
num_classes = 2
net = EfficientNet(
    version = version,
    num_classes = num_classes,
)

In [ ]:
criterion = nn.CrossEntropyLoss() # for classification problems

# lr , weight_decay  --> 하이퍼 파라미터
optimizer = optim.Adam(net.parameters(), lr = 0.001, weight_decay = 1e-4)

In [ ]:
net.train()

# epoch 하이퍼 파라미터

for epoch in range(200) : 
  total_correct = 0.0
  running_loss = 0.0
  for i, (inputs, labels) in enumerate(train_set) : 
    inputs, labels = inputs, labels
    output = net(inputs)
    output_idx = torch.argmax(output, dim=1)  # input 텐서 에있는 모든 요소의 최대 값 인덱스를 반환, dim = 0 은 열 기준, dim = 1은 행 기준
    total_correct += (labels ==output_idx).sum().item()
    optimizer.zero_grad()
    loss = criterion(output, labels)
    running_loss += loss.item() * inputs.size(0)
    loss.backward()
    optimizer.step()
  print(f'Epoch: {epoch}  Loss: {running_loss/train_len}  Accuracy: {(total_correct/train_len) * 100}%')

print('Finished training')

Epoch: 0  Loss: 0.7801760921633341  Accuracy: 53.82113821138211%
Epoch: 1  Loss: 0.731478610465197  Accuracy: 49.105691056910565%
Epoch: 2  Loss: 0.7115134769338902  Accuracy: 52.357723577235774%
Epoch: 3  Loss: 0.7018137395866518  Accuracy: 51.707317073170735%
Epoch: 4  Loss: 0.7052098343042823  Accuracy: 50.89430894308943%
Epoch: 5  Loss: 0.7023706421619509  Accuracy: 53.98373983739837%
Epoch: 6  Loss: 0.698658280353236  Accuracy: 54.63414634146342%
Epoch: 7  Loss: 0.6937803823773454  Accuracy: 53.65853658536586%
Epoch: 8  Loss: 0.7095885078112284  Accuracy: 53.170731707317074%
Epoch: 9  Loss: 0.7073187885245656  Accuracy: 47.3170731707317%
Epoch: 10  Loss: 0.6903741856900657  Accuracy: 55.1219512195122%
Epoch: 11  Loss: 0.6925318857518638  Accuracy: 53.170731707317074%
Epoch: 12  Loss: 0.6945358671793124  Accuracy: 53.98373983739837%
Epoch: 13  Loss: 0.695380706031148  Accuracy: 54.3089430894309%
Epoch: 14  Loss: 0.6919302716487792  Accuracy: 53.82113821138211%
Epoch: 15  Loss: 0.69

In [ ]:
from sklearn import metrics

y_true = []
y_pred = []

with torch.no_grad() : 
  net.eval()  # evaluation 과정에서 사용하지 않아야 하는 layer들을 알아서 off 시키도록 하는 함수(보통 dropout, batchnorm layer)
  total_loss = 0.0
  total_correct = 0.0

  for inputs, labels in test_set : 
    outputs = net(inputs)
    preds = torch.max(outputs, 1)
    loss = criterion(outputs, labels)
    total_loss += loss.item() * inputs.size(0)
    output_idx = torch.argmax(outputs, dim = 1)
    total_correct +=sum(labels == output_idx)
    for i in range(len(labels.data.tolist())) : 
      y_true.append(labels.data.tolist()[i])
      y_pred.append(preds.indices.tolist()[i])

# 한 배치의 첫 번째 이미지에 대하여 결과 시각화
# print(f'[예측 결과: {class_names[preds[0]]}] (실제 정답: {class_names[labels.data[0]]})')
# imshow(inputs.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

epoch_loss = total_loss / test_len
epoch_acc = total_correct / test_len * 100.
print('[Test Phase] Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
print('precision:', metrics.precision_score(y_true, y_pred))
print('recall:', metrics.recall_score(y_true, y_pred))
print('f1_score:', metrics.f1_score(y_true, y_pred))

[Test Phase] Loss: 1.5658 Acc: 58.2278
precision: 0.5833333333333334
recall: 0.5384615384615384
f1_score: 0.5599999999999999


모델 저장

In [ ]:
torch.save(net.state_dict(), f'{patH}/Model/38.pt')